# Aggregation with dynamic features
This notebook shows how to do the aggregation with monthly values for certain columns. 

__Remark:__ Because we now filter out a lot of users that only visited once, this notebook is not such a pain in the ass anymore. Don't be afraid to run it, your memory will be sufficient and you'll be done in a couple of minutes.

In [1]:
import json
import datetime
import os
import time
import sys
import shutil
import glob
import re

import pandas as pd
import numpy as np
from sklearn import preprocessing

import matplotlib.pyplot as plt

sys.path.append('..')
from preprocessing import *
from aggregation import *

## Managing a huge file

Below is the new version of `load`, where processing takes place in chunks. After all chunks have been processed, they are concatenated to a single file. Since many columns are either dropped or aggregated, the resulting dataframe fits in RAM.

In [88]:
def reduce_df(path, output, nrows=None, chunksize=20000):
    """ Load Google analytics data from JSON into a Pandas.DataFrame. """
    if nrows and chunksize:
        msg = "Reading {} rows in chunks of {}. We are gonna need {} chunks"
        print(msg.format(nrows, chunksize, nrows / chunksize))
    
    temp_dir = "../data/temp"
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    i = 0
    for chunk in pd.read_csv(path, 
                             converters={column: json.loads for column in JSON_COLUMNS},
                             dtype={'fullVisitorId': 'str'},
                             nrows=nrows,
                             chunksize=chunksize):
        
        chunk = chunk.reset_index()
        
        # Normalize JSON columns
        for column in JSON_COLUMNS:
            column_as_df = pd.io.json.json_normalize(chunk[column])
            chunk = chunk.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        # Parse date
        chunk['date'] = chunk['date'].apply(lambda x: pd.datetime.strptime(str(x), '%Y%m%d'))
        
        # Only keep relevant columns
        cols = ['date', 'fullVisitorId', 'operatingSystem', 'country', 'browser',
                'pageviews', 'transactions', 'visits', 'transactionRevenue']
        try:
            chunk = chunk[cols]
        except KeyError as e:
            # Regex magic to find exactly which columns were not found.
            # Might be different in Python 3, be careful!
            missing_cols = list(re.findall(r"'(.*?)'", e.args[0]))
            for col in missing_cols:
                print("Column {} was not found in chunk {}, filling with zeroes".format(col, i))
                chunk[col] = [0] * len(chunk)
            chunk = chunk[cols]
            
        
        print("Loaded chunk {}, Shape is: {}".format(i, chunk.shape))
        chunk.to_csv(os.path.join(temp_dir, str(i) + ".csv"), encoding='utf-8', index=False)
        i += 1
    
    print("Finished all chunks, now concatenating")
    files = glob.glob(os.path.join(temp_dir, "*.csv"))
    with open(output, 'wb') as outfile:
        for i, fname in enumerate(files):
            with open(fname, 'rb') as infile:
                # Throw away header on all but first file
                if i != 0:
                    infile.readline()  
                # Block copy rest of file from input to output without parsing
                shutil.copyfileobj(infile, outfile)

reduce_df("../data/train_v2.csv", output="../data/reduced_train.csv", nrows=None, chunksize=20000)
reduce_df("../data/test_v2.csv", output="../data/reduced_test.csv", nrows=None, chunksize=20000)

Loaded chunk 0, |Shape is: (20000, 9)
Loaded chunk 1, |Shape is: (20000, 9)
Loaded chunk 2, |Shape is: (20000, 9)
Loaded chunk 3, |Shape is: (20000, 9)
Loaded chunk 4, |Shape is: (20000, 9)
Loaded chunk 5, |Shape is: (20000, 9)
Loaded chunk 6, |Shape is: (20000, 9)
Loaded chunk 7, |Shape is: (20000, 9)
Loaded chunk 8, |Shape is: (20000, 9)
Loaded chunk 9, |Shape is: (20000, 9)
Loaded chunk 10, |Shape is: (20000, 9)
Loaded chunk 11, |Shape is: (20000, 9)
Loaded chunk 12, |Shape is: (20000, 9)
Loaded chunk 13, |Shape is: (20000, 9)
Loaded chunk 14, |Shape is: (20000, 9)
Loaded chunk 15, |Shape is: (20000, 9)
Loaded chunk 16, |Shape is: (20000, 9)
Loaded chunk 17, |Shape is: (20000, 9)
Loaded chunk 18, |Shape is: (20000, 9)
Loaded chunk 19, |Shape is: (20000, 9)
Loaded chunk 20, |Shape is: (20000, 9)
Loaded chunk 21, |Shape is: (20000, 9)
Loaded chunk 22, |Shape is: (20000, 9)
Loaded chunk 23, |Shape is: (20000, 9)
Loaded chunk 24, |Shape is: (20000, 9)
Loaded chunk 25, |Shape is: (20000,

Loaded chunk 0, |Shape is: (20000, 9)
Loaded chunk 1, |Shape is: (20000, 9)
Loaded chunk 2, |Shape is: (20000, 9)
Loaded chunk 3, |Shape is: (20000, 9)
Loaded chunk 4, |Shape is: (20000, 9)
Loaded chunk 5, |Shape is: (20000, 9)
Loaded chunk 6, |Shape is: (20000, 9)
Loaded chunk 7, |Shape is: (20000, 9)
Loaded chunk 8, |Shape is: (20000, 9)
Loaded chunk 9, |Shape is: (20000, 9)
Loaded chunk 10, |Shape is: (20000, 9)
Loaded chunk 11, |Shape is: (20000, 9)
Loaded chunk 12, |Shape is: (20000, 9)
Loaded chunk 13, |Shape is: (20000, 9)
Loaded chunk 14, |Shape is: (20000, 9)
Loaded chunk 15, |Shape is: (20000, 9)
Loaded chunk 16, |Shape is: (20000, 9)
Loaded chunk 17, |Shape is: (20000, 9)
Loaded chunk 18, |Shape is: (20000, 9)
Loaded chunk 19, |Shape is: (20000, 9)
Loaded chunk 20, |Shape is: (1589, 9)
Finished all chunks, now concatenating


In [98]:
train = pd.read_csv("../data/reduced_train.csv")
train['transactions'].fillna(0, inplace=True)
train['transactionRevenue'].fillna(0.0, inplace=True)

def agg_date(s):
    date = pd.datetime.strptime(x, '%Y-%m-%d')
    return "{}_{}".format(x.month, x.year)
        
train["date"] = train["date"].apply(agg_date)


In [109]:
most_frequent = lambda x:x.value_counts().index[0]
agg = {
    "operatingSystem": most_frequent,
    "country": most_frequent,
    "browser": most_frequent,
    "pageviews": sum,
    "transactions": sum,
    "visits": sum,
    "transactionRevenue": sum
}
grouped = train.groupby(["fullVisitorId", "date"], as_index=False).agg(agg)

In [112]:
grouped.to_csv("../data/grouped.csv", encoding='utf-8', index=False)

### Load and preprocess the data
First we need to load and preprocess the original data.
Note that there might be three additional columns in the new dataset. These need to be preprocessed as well.

The `preprocess_and_save` method now has an argument `drop_users=True` (default is True). You can set this to false if you wish to keep all users.

In [ ]:
preprocess_and_save("../data", nrows_train=None, nrows_test=None, start_x_train='2016-08-01', 
                    end_x_train='2016-10-16', start_y_train='2016-12-01', end_y_train='2017-02-01', 
                    start_x_test='2017-08-01', end_x_test='2017-10-16', drop_users=True)

### Aggregating data per customer

We need to predict the target at customer-level, i.e., we predict one value for each customer
in the test set. Our data, however, contains a row for every site visit. One obvious way to deal
with this discrepancy is to aggregate the visit data by customer, to obtain features on the
customer-level.

The _preprocessing.py_ file contains functions to do so.

In [ ]:
data_dir = "../data/"
x_train, y_train, x_test = load_train_test_dataframes(data_dir, nrows_train=None, nrows_test=None)

In [ ]:
print("This only takes a few seconds now.")
x_train_aggregated = aggregate_data_per_customer(x_train, startdate_y='2016-12-01', startdate_x='2016-08-01')
print("Train data is aggregated")
print("Aggregating test data.")
x_test_aggregated = aggregate_data_per_customer(x_test, startdate_y='2017-12-01', startdate_x='2017-08-01')
print("Test data is aggregated")
y_train_aggregated = y_train.groupby(['fullVisitorId'])[['target']].sum()
print("Train target data is aggregated")

This data should be close to what we need to start fitting models. We tried to keep as much information as possible, so from here it is of course still possible to remove features that seem unnecessary or do other dimensionality reduction. The good thing is that we don't have to do aggregation every time, so let's save the results:

In [ ]:
print("Saving files")
x_train_aggregated.to_csv(os.path.join(data_dir, "aggregated_x_train.csv"), index=True)
x_test_aggregated.to_csv(os.path.join(data_dir, "aggregated_x_test.csv"), index=True)
y_train_aggregated.to_csv(os.path.join(data_dir, "aggregated_y_train.csv"), index=True)
print("Aggregated data saved")

### Our first attempt!
Let's see if we can fit a model on this data.

Note that due to the one-hot encoding, the columns of train and test are not the same. For this experiment, only keep the intersection of columns. There are also other ways to deal with this (e.g., by mapping categories to external data), so we don't do this in the aggregation step.

In [ ]:
# just for illustration, so let's keep it simple
from sklearn import linear_model

# create train and test sets and labels excluding visitor ID
x_train, x_test = keep_intersection_of_columns(aggregated_train.reset_index(drop=True),
                                               aggregated_test.reset_index(drop=True))
y_train = np.log(aggregated_train.reset_index(drop=True)["target_sum"]+1)

In [ ]:
# set NaNs to zero and fit linear model
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
r_squared = lm.score(x_train, y_train)
print("The model has an R^2 of {}.".format(r_squared))

In [ ]:
# predict and create a submission
predictions = lm.predict(x_test)
submission = pd.concat([aggregated_test.reset_index()["fullVisitorId"], pd.Series(predictions)], axis=1)
submission.columns = ["fullVisitorId", "PredictedLogRevenue"]

# set everything below $1 to zero
submission["PredictedLogRevenue"] = np.maximum(0, submission["PredictedLogRevenue"])
submission["PredictedLogRevenue"][submission["PredictedLogRevenue"]<1] = 0
submission.head()

In [ ]:
submission.to_csv("first_submission.csv", index=False)